In [1]:
import keras
import pandas as pd
df = pd.read_csv(r'C:\Users\hopio\Desktop\공부자료\kmooc 크롤링\sentiment labelled sentences\amazon_cells_labelled.txt', sep='\t', header=None)
df.head()



,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [3]:
from keras.preprocessing.text import Tokenizer


tok = Tokenizer()

tok.fit_on_texts(df[0])
tok.word_index['plug'],tok.index_word[155]

(155, 'plug')

In [5]:
seq = tok.texts_to_sequences(df[0])
seq[0]

[33,
 117,
 5,
 53,
 214,
 11,
 47,
 8,
 155,
 4,
 19,
 337,
 19,
 1,
 546,
 416,
 2,
 241,
 190,
 6,
 812]

In [8]:
text = tok.sequences_to_texts(seq)
text[0] #=' '.join(tok.index_word[i] for i in seq[0])

'so there is no way for me to plug it in here in the us unless i go by a converter'

In [31]:
#  correct import
from keras.utils import pad_sequences

' '.join(tok.index_word[i] for i in seq[0])

MAXLEN = max(len(s) for s in seq)
pad = pad_sequences(seq, MAXLEN)
pad[0]


array([  0,   0,   0,   0,   0,   0,   0,   0,   0,  33, 117,   5,  53,
       214,  11,  47,   8, 155,   4,  19, 337,  19,   1, 546, 416,   2,
       241, 190,   6, 812])

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(pad, df[1], test_size=.2, random_state=1234)


NUM_WORDS = len(tok.index_word) + 1
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

rnn = Sequential()
rnn.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN, mask_zero=True))
rnn.add(LSTM(16, return_sequences=False))
rnn.add(Dense(1, activation='sigmoid'))
rnn.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 30, 8)             15032     
                                                                 
 lstm_5 (LSTM)               (None, 16)                1600      
                                                                 
 dense_6 (Dense)             (None, 1)                 17        
                                                                 
Total params: 16,649
Trainable params: 16,649
Non-trainable params: 0
_________________________________________________________________


In [33]:
from keras.optimizers import Adam
rnn.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
rnn.fit(X_train, y_train, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 5ms/step - loss: 0.6931 - accuracy: 0.4825
Epoch 2/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6879 - accuracy: 0.7113
Epoch 3/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6677 - accuracy: 0.7887
Epoch 4/10
25/25 [==============================] - 0s 5ms/step - loss: 0.5656 - accuracy: 0.8537
Epoch 5/10
25/25 [==============================] - 0s 5ms/step - loss: 0.4249 - accuracy: 0.8975
Epoch 6/10
25/25 [==============================] - 0s 5ms/step - loss: 0.3201 - accuracy: 0.9413
Epoch 7/10
25/25 [==============================] - 0s 5ms/step - loss: 0.2413 - accuracy: 0.9625
Epoch 8/10
25/25 [==============================] - 0s 5ms/step - loss: 0.2122 - accuracy: 0.9650
Epoch 9/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1661 - accuracy: 0.9750
Epoch 10/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1312 - accuracy: 0.9837


In [34]:
y_rnn = (rnn.predict(X_test) > 0.5)*1

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_rnn)

7/7 [==============================] - 1s 2ms/step


0.79

In [35]:
# - 역방향 
# 단어처리 순서를 앞에서 뒤로 하는 것이 아니라 역방향으로 뒤에서 앞으로도 할 수 있다. 순환신경망 레이어에 go_backwards=True를 추가해주면 된다.
rnn_goback = Sequential()
rnn_goback.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN, mask_zero=True))
rnn_goback.add(LSTM(16, return_sequences=False, go_backwards=True))
rnn_goback.add(Dense(1, activation='sigmoid'))

rnn_goback.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
rnn_goback.fit(X_train, y_train, epochs=10)




Epoch 1/10
25/25 [==============================] - 2s 5ms/step - loss: 0.6930 - accuracy: 0.5000
Epoch 2/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6876 - accuracy: 0.5612
Epoch 3/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6676 - accuracy: 0.8300
Epoch 4/10
25/25 [==============================] - 0s 5ms/step - loss: 0.5835 - accuracy: 0.8562
Epoch 5/10
25/25 [==============================] - 0s 6ms/step - loss: 0.4256 - accuracy: 0.9087
Epoch 6/10
25/25 [==============================] - 0s 6ms/step - loss: 0.3139 - accuracy: 0.9400
Epoch 7/10
25/25 [==============================] - 0s 5ms/step - loss: 0.2460 - accuracy: 0.9575
Epoch 8/10
25/25 [==============================] - 0s 7ms/step - loss: 0.2003 - accuracy: 0.9700
Epoch 9/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1736 - accuracy: 0.9750
Epoch 10/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1615 - accuracy: 0.9725


In [37]:
y_rnn_goback = (rnn_goback.predict(X_test) > 0.5)*1

from sklearn.metrics import accuracy_score
accuracy_score(y_rnn_goback, y_rnn)

7/7 [==============================] - 0s 3ms/step


0.725

In [38]:
 # - 양방향
# 순방향 RNN과 양방향 RNN을 합치면 양방향 RNN이 된다. Bidirectional을 사용한다.
from keras.layers import Bidirectional
rnn_bidirec = Sequential()
rnn_bidirec.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN, mask_zero=True))
rnn_bidirec.add(Bidirectional(LSTM(16, return_sequences=False)))
rnn_bidirec.add(Dense(1, activation='sigmoid'))


rnn_bidirec.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
rnn_bidirec.fit(X_train, y_train, epochs=10)




Epoch 1/10
25/25 [==============================] - 4s 5ms/step - loss: 0.6931 - accuracy: 0.5013
Epoch 2/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6897 - accuracy: 0.5425
Epoch 3/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6775 - accuracy: 0.7950
Epoch 4/10
25/25 [==============================] - 0s 5ms/step - loss: 0.6116 - accuracy: 0.8350
Epoch 5/10
25/25 [==============================] - 0s 6ms/step - loss: 0.3938 - accuracy: 0.9075
Epoch 6/10
25/25 [==============================] - 0s 5ms/step - loss: 0.2409 - accuracy: 0.9600
Epoch 7/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1659 - accuracy: 0.9750
Epoch 8/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1274 - accuracy: 0.9787
Epoch 9/10
25/25 [==============================] - 0s 5ms/step - loss: 0.1016 - accuracy: 0.9875
Epoch 10/10
25/25 [==============================] - 0s 5ms/step - loss: 0.0882 - accuracy: 0.9875


In [39]:
y_rnn_bidirec = (rnn_bidirec.predict(X_test) > 0.5)*1

from sklearn.metrics import accuracy_score
accuracy_score(y_rnn_bidirec, y_rnn)


7/7 [==============================] - 2s 3ms/step


0.87

In [40]:
# CNN 모델 추가

from keras.layers import Conv1D, GlobalMaxPooling1D


cnn = Sequential()
cnn.add(Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN))
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
cnn.add(GlobalMaxPooling1D())
cnn.add(Dense(1, activation='sigmoid'))

cnn.summary()

# 모델 컴파일 및 학습
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
cnn.fit(X_train, y_train, epochs=10)

# 테스트 데이터 평가
accuracy = cnn.evaluate(X_test, y_test)[1]
print("Test Accuracy:", accuracy)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 30, 8)             15032     
                                                                 
 conv1d_2 (Conv1D)           (None, 28, 16)            400       
                                                                 
 global_max_pooling1d_2 (Glo  (None, 16)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 15,449
Trainable params: 15,449
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6938 - accuracy: 0.4900
Epoch 2/10
25/25 [

In [41]:
# 앙상블 모델 = LSTM 모델 + CNN 모델

from keras.models import Model
from keras.layers import Input, concatenate


lstm_input = Input(shape=(MAXLEN,))
lstm_embedding = Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN)(lstm_input)
lstm_output = LSTM(16, return_sequences=False)(lstm_embedding)

# CNN 모델
cnn_input = Input(shape=(MAXLEN,))
cnn_embedding = Embedding(input_dim=NUM_WORDS, output_dim=8, input_length=MAXLEN)(cnn_input)
cnn_conv = Conv1D(filters=16, kernel_size=3, activation='relu')(cnn_embedding)
cnn_pool = GlobalMaxPooling1D()(cnn_conv)

# LSTM과 CNN 모델 병합
merged = concatenate([lstm_output, cnn_pool])
output = Dense(1, activation='sigmoid')(merged)

# 앙상블 모델 생성
ensemble_model = Model(inputs=[lstm_input, cnn_input], outputs=output)
ensemble_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

ensemble_model.summary()

# 학습 및 평가
ensemble_model.fit([X_train, X_train], y_train, epochs=10)
accuracy = ensemble_model.evaluate([X_test, X_test], y_test)[1]
print("Test Accuracy:", accuracy)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 embedding_13 (Embedding)       (None, 30, 8)        15032       ['input_4[0][0]']                
                                                                                                  
 embedding_12 (Embedding)       (None, 30, 8)        15032       ['input_3[0][0]']                
                                                                                            